## Ridge & Lasso Regularization techniques
In this notebook I'll attempt to test out Ridge & Lasso algorithms.


### Ridge Regression
<b>When to use it?</b> When we create a Linear Regression model and want to make it more accurate on the test dataset.
Therefore, we sacrifice the accuracy of the training set to make it predict better on the test data. Thus, making better
predictions in the long-term.(Also known as bias-variance tradeoff)

- Linear Regression reduces the <b>sum of squared error/residuals</b>, while Ridge reduces the <b>sum of squared residuals + penalty*magnitude</b>
- Can be used in both linear & logistic regressions
- Uses L2 regularization

<b>Formula</b>: $$SSR + \lambda * slope^2$$, where SSR is <b>sum of squared residuals</b>; $$\lambda$$ is the
<b>magnitude</b> of penalty; <b>slope</b> is the penalty itself

### Important!
Regularization doesn't require any preprocessing steps <b>except standardization</b>. It works by enriching the
learning process using a penalization for too complex models to shrink (or reduce to zero)
coefficients relative to variables that are irrelevant for your prediction term or are
redundant, as they are highly correlated with others present in the model.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

%matplotlib inline
sns.set()

In [8]:
df = load_boston()
boston = pd.DataFrame(df.data, columns=df.feature_names)
boston['Price'] = df.target

X = boston.iloc[:, :-1]
y = boston.iloc[:, -1]


In [15]:
def results(algorithm, train, train_target, test, test_target, text=''):
    print(text.upper())
    # calculate score
    print(f'In-sample score: {algorithm.score(train, train_target)}')
    print(f'Out-sample score: {algorithm.score(test, test_target)}')

    print('\n==================\n')

    # calculate MSE (mean squared error)
    train_pred = algorithm.predict(train)
    test_pred = algorithm.predict(test)
    print(f'In-Sample MSE: {mean_squared_error(train_target, train_pred)}')
    print(f'Out-sample MSE: {mean_squared_error(test_target, test_pred)}')

    print('\n==================\n')

    # calculate RSS (residual sum squared)
    print(f'In-sample RSS: {np.sum(np.square(train_target - train_pred))}')
    print(f'Out-sample RSS: {np.sum(np.square(test_target - test_pred))}\n')

# linear regression first
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
linear = LinearRegression().fit(X_train, y_train)
results(linear, X_train, y_train, X_test, y_test, text='unstandardized')


UNSTANDARDIZED
In-sample score: 0.7434997532004697
Out-sample score: 0.7112260057484908


In-Sample MSE: 22.545481487421426
Out-sample MSE: 21.51744423117739


In-sample RSS: 7981.100446547185
Out-sample RSS: 3270.651523138963



In [16]:
# standardize & check results
boston_stand = StandardScaler().fit_transform(boston)
Xst = boston_stand[:, :-1]
yst = boston_stand[:, -1]

Xst_train, Xst_test, yst_train, yst_test = train_test_split(Xst, yst, test_size=0.3, random_state=42)
linear_standardized = LinearRegression(normalize=True).fit(Xst_train, yst_train)
results(linear_standardized, Xst_train, yst_train, Xst_test, yst_test, text='standardized')

STANDARDIZED
In-sample score: 0.7434997532004697
Out-sample score: 0.7112260057484932


In-Sample MSE: 0.2670646768826304
Out-sample MSE: 0.25488696234522545


In-sample RSS: 94.54089561645117
Out-sample RSS: 38.74281827647427



In [21]:
# ridge
ridge = Ridge(alpha=1).fit(Xst_train, yst_train)
results(ridge, Xst_train, yst_train, Xst_test, yst_test, text='Ridge standardized')

RIDGE STANDARDIZED
In-sample score: 0.7434821404219876
Out-sample score: 0.7108092176450824


In-Sample MSE: 0.26708301507549
Out-sample MSE: 0.2552548412253697


In-sample RSS: 94.54738733672346
Out-sample RSS: 38.79873586625619



In [31]:
ridge_cv = RidgeCV(alphas=(0.01, 0.1, 1.0, 10.0), cv=5, store_cv_values=True)